## Tool to scrap BPM from https://songbpm.com

### prerequisites
```bash
conda install -c rogerramos bs4
```

In [68]:
import requests
from bs4 import BeautifulSoup

def parse_token(content):
    soup = BeautifulSoup(content, 'lxml')
    return soup.find('input', {'name':'_csrf'})['value']
    
def parse_bpm(content):
    soup = BeautifulSoup(content, 'lxml')
    first_card = soup.find('div', { 'class' : 'card' })
    if not first_card:
        return None
    media_right = first_card.find('div', { 'class' : 'media-right' })
    titles = media_right.findAll('p', { 'class' : 'title' })
    return int(titles[1].text)

def scrap_bpm(artist = '', song = ''):
    with requests.Session() as s:
        r1 = s.get('https://songbpm.com')
        token = parse_token(r1.text)
        r2 = s.post('https://songbpm.com/searches', data = {'_csrf': token, 'q': '%s %s'%(artist, song)})
        return parse_bpm(r2.text)

In [37]:
scrap_bpm('the offspring', 'conspiracy of one')

106

In [69]:
import pandas as pd
df = pd.read_csv('../Data/billboard.csv', encoding='latin1')
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
artist_track = df[['artist.inverted', 'track']]
artist_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 2 columns):
artist.inverted    317 non-null object
track              317 non-null object
dtypes: object(2)
memory usage: 5.0+ KB


In [ ]:
import time

def process(df):
    time.sleep(10)
    bpm = scrap_bpm(df['artist.inverted'], df['track'])
    df['bpm'] = bpm
    print('%s, %s, %s'%(df['artist.inverted'], df['track'], str(bpm)))
    return df

artist_track = artist_track.apply(process, axis = 1)

Destiny's Child, Independent Women Part I, 98
Destiny's Child, Independent Women Part I, 98
Santana, Maria, Maria, 98
Savage Garden, I Knew I Loved You, 170
Madonna, Music, 120
Aguilera, Christina, Come On Over Baby (All I Want Is You), 119
Janet, Doesn't Really Matter, 99
Destiny's Child, Say My Name, 138
Iglesias, Enrique, Be With You, 122
Sisqo, Incomplete, 119
Lonestar, Amazed, 140
N'Sync, It's Gonna Be Me, 165
Aguilera, Christina, What A Girl Wants, 142
Vertical Horizon, Everything You Want, 94
Creed, With Arms Wide Open, 139
Aaliyah, Try Again, 95
matchbox twenty, Bent, 95
Carey, Mariah, Thank God I Found You, 130
Hill, Faith, Breathe, 137
Mya, Case Of The Ex (Whatcha Gonna Do), 98
Dream, He Loves U Not, 156
Braxton, Toni, He Wasn't Man Enough, 88
Anthony, Marc, You Sang To Me, 166
98¡, Give Me Just One Night (Una Noche), 124
Destiny's Child, Jumpin' Jumpin', 89
3 Doors Down, Kryptonite, 99
Sisqo, Thong Song, 122
Aguilera, Christina, I Turn To You, 127
Houston, Whitney, My Love I

Lil Wayne, Tha Block Is Hot, 174
2 Pac, Baby Don't Cry (Keep Ya Head Up II), 94
Wallflowers, The, Sleepwalker, 129
Drama, Left, Right, Left, 125
Limp Bizkit, N 2 Gether Now, 102
Lox, Ryde or Die, Chick, None
Ying Yang Twins, Whistle While You Twurk, 102
Walker, Clay, Live, Laugh, Love, 141
Metallica, No Leaf Clover (Live), 151
Yankee Grey, Another Nine Minutes, 138
Bega, Lou, Tricky Tricky , 151
Big Punisher, It's So Hard, 93
Worley, Darryl, When You Need My Love, 86
Lil' Kim, How Many Licks?, 105
Clark, Terri, A Little Gasoline, 77
Brock, Chad, A Country Boy Can Survive, 76
Cagle, Chris, My Love Goes On And On, 162
Byrd, Tracy, Put Your Hand In Mine, 72
Metallica, I Disappear, 123
Jay-Z, Hey Papi, 158
Fatboy Slim, The Rockafeller Skank, 153
Levert, Gerald, Mr. Too Damn Good, 115
McKnight, Brian, Stay Or Let It Go, 80
Allan, Gary, Smoke Rings In The Dark, 83
Ginuwine, The Best Man I Can Be, 112
Before Dark, Monica, 88
Estefan, Gloria, No Me Dejes De Querer, 83
Stone Temple Pilots, Sour

In [ ]:
artist_track

In [73]:
artist_track.to_csv('./bpm.csv', encoding='latin1', index = False)

In [3]:
import pandas as pd
x = pd.read_csv('./bpm.csv', encoding='latin1')

x[pd.isnull(x).any(axis=1)]

,artist.inverted,track,bpm
35,"Elliott, Missy ""Misdemeanor""",Hot Boyz,NaN
44,"Badu, Erkyah",Bag Lady,NaN
45,Pink,There U Go,NaN
49,Kenny G,Auld Lang Syne (The Millenium Mix),NaN
225,Hot Boys,I Need A Hot Girl,NaN
226,Jay-Z,Do It Again (Put Ya Hands Up),NaN
250,Lox,"Ryde or Die, Chick",NaN
286,"Carter, Torrey",Take That,NaN
297,Funkmaster Flex,Do You,NaN
303,Lil' Mo,Ta Da,NaN
